In [1]:
import requests
from bs4 import BeautifulSoup as BS #pip install bs4 --user

In [2]:
import telebot
import pandas as pd
import facebook
import re
import requests
from lxml import etree

In [3]:
access_token = 'EAACEdEose0cBAFkaNvCactG2NfAEQhZAo0PnDoLrtajXWhnDu2qPNEEbQtNeW8B23tqU9wwOCeFjcLV7X9KrwpxAUf616pYUK2MfmJ9RVAI1RZCYYx0ktnsugQ0veLVpXAwAuOxvtBOzAPgIjjlTDZCR5QZAXYfX7hNC9n7CPrUyV0hG3nPaQwVIYNleXiEZD'

In [4]:
def fb_pasrsing(access_token,chat_id):
    graph = facebook.GraphAPI(access_token)

    # id of 'Flats for friends' page
    page = 'afisha'

    profile = graph.get_object(page)
    posts = graph.get_connections(profile['id'], 'feed')['data']

    try:
        seen_posts = list(pd.read_csv('seen_posts_afisha'+str(chat_id)+'.csv').post_id)
    except:
        seen_posts = []
        file = open('seen_posts_afisha'+str(chat_id)+'.csv', 'w')
        file.write('post_id')
        file.close()
        
    seen_posts_new, posts_to_show = [], []
    for post in posts:
        if len(set(seen_posts)&{post['id']})==0:
            seen_posts_new.append(post['id'])
            flag = False
            try:
                links = re.findall('http://.+ ?', post['message'])
                for link in links:
                    article = []
                    page = requests.get(link).text
                    soup = BS(page, 'lxml')
                    news = soup.find_all('div', {'class':'news-paragraph'}) #article paragraph
                    if len(news) > 0:
                        for i in news:
                            ps = i.find_all('p')
                            for p in ps:
                                contents = p.contents
                                for cont in contents:
                                    cont = str(cont)
                                    if 'фильм' in cont:
                                        flag = True
                        if flag == True:
                            posts_to_show.append(link)
                    else:
                        news = soup.find_all('div', {'class':'article-paragraph'}) #article paragraph
                        for i in news:
                            ps = i.find_all('p')
                            for p in ps:
                                contents = p.contents
                                for cont in contents:
                                    cont = str(cont)
                                    if 'фильм' in cont:
                                        flag = True
                        if flag == True:
                            posts_to_show.append(link)
                        
            except KeyError:
                pass
    pd.DataFrame(seen_posts+seen_posts_new,columns=['post_id']).to_csv('seen_posts_afisha'+str(chat_id)+'.csv', index=False) 
    return(posts_to_show)

In [5]:
tg_token = '273216415:AAHnRdf3P_tio6OstPekkwRcdu-xF5USe84'

In [8]:
bot = telebot.TeleBot(tg_token)

@bot.message_handler(commands=['start'])

def handle_start(message):
   bot.send_message(message.chat.id, "Hi!")
    
@bot.message_handler(commands=['search_films'])

def search_flats(message):
    bot.send_message(message.chat.id, 'Ok, wait, I am working on it...')
    posts_to_show = fb_pasrsing(access_token, message.chat.id)
    if len(posts_to_show)==0:
        bot.send_message(message.chat.id, 'Since the last request, there were no new posts, sorry :(')
    else:
        for link in posts_to_show:
            bot.send_message(message.chat.id, link)
        bot.send_message(message.chat.id, 'Done!')

@bot.message_handler(commands=['help'])
def handle_help(message):
    bot.send_message(message.chat.id, """
/start - say Hi!
/search_films - search for new posts about films
""")
            
if __name__ == '__main__':
   bot.polling(none_stop=True)